INSTRUCTIONS:
- Run setup
- Upload papers.parquet from data/scientific/processed
- **Run Nov 8 part**

# SETUP

In [ ]:
# !pip install torch transformers datasets sentencepiece tqdm scipy sentence-transformers datasets evaluate scikit-learn evaluate

In [ ]:
from sentence_transformers import SentenceTransformer, models, losses, InputExample, evaluation
import torch
import os
from datasets import load_dataset
import numpy as np
from typing import Tuple
from torch import nn
from datasets import load_dataset, DatasetDict, load_from_disk # Import load_from_disk
from sentence_transformers.readers import STSBenchmarkDataReader
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import evaluate

Architecture:
* Encoder
  * word_embedding_model generates word embeddings from text.
  * in BERT, sentence embeding is based on
    1) token embedding (words)
    2) segment embedding
    3) position embedding (words order)
  * spits out sentence embeddings

* Pooler
  * FC layer
  * performs task given embedding from ecnoder

Training: train encoder with different pooler dimensions and select best encodere for this task.

Note: Ctrl+F ABOBA to change dims

# DATA

In [ ]:
# Assuming the parquet file is in the content directory
parquet_file_path = '/content/papers.parquet'

try:
    df_papers = pd.read_parquet(parquet_file_path, engine='pyarrow') # or engine='fastparquet'
    print("Parquet file loaded successfully!")
    display(df_papers.head())
except FileNotFoundError:
    print(f"Error: The file {parquet_file_path} was not found.")
except Exception as e:
    print(f"An error occurred while loading the parquet file: {e}")

Parquet file loaded successfully!


,corpus,doc_id,sent_index,sentence,split,token_count
0,scientific_papers,t_sne_exaggerates_clusters_provably__related_work,0,There are two notably distinct ways of context...,train,16
1,scientific_papers,t_sne_exaggerates_clusters_provably__related_work,1,In a chronological sense.,train,4
2,scientific_papers,t_sne_exaggerates_clusters_provably__related_work,2,Confidence in the data visualizations produced...,train,20
3,scientific_papers,t_sne_exaggerates_clusters_provably__related_work,3,Some works argue that these methods have merit...,train,31
4,scientific_papers,t_sne_exaggerates_clusters_provably__related_work,4,Along with these algorithmic performance guara...,train,27


In [ ]:
# from sklearn.model_selection import train_test_split

# # Assuming df_papers is your DataFrame and 'sentence' is the column you want to use
# # You might want to select specific columns before splitting depending on your task
# X = df_papers['sentence'] # Features
# # If you have a target variable, include it here. Otherwise, you might just split X.
# # For sentence embedding, we might not have a direct 'y' target for the initial split.
# # If you need to split based on some other column, replace X accordingly.

# # Splitting the data (e.g., 80% train, 20% test)
# # Adjust test_size as needed
# X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# print("Original data size:", len(X))
# print("Training set size:", len(X_train))
# print("Testing set size:", len(X_test))

# # Display the first few items of the training set
# print("\nFirst 5 items of the training set:")
# display(X_train.head())

Original data size: 32
Training set size: 25
Testing set size: 7

First 5 items of the training set:


,sentence
25,"From a theoretical perspective, the tradeoff b..."
12,"As t-SNE, UMAP (developed circa 2008 and 2018,..."
0,There are two notably distinct ways of context...
4,Along with these algorithmic performance guara...
16,Building upon the precision-recall framework of.


## IMDB

This is sentiment analysis, will be using later.

In [ ]:
# import pandas as pd

# # imdb is a bindary classification dataset from Hugging Face

# url = 'https://raw.githubusercontent.com/AlenaResiko/UML/hugging_face/data/huggingface/imdb/train.parquet'
# !wget -O train.parquet "$url"
# df_imdb_train = pd.read_parquet('train.parquet', engine='pyarrow')  # or engine='fastparquet'
# df_imdb_train.head()

# url = 'https://raw.githubusercontent.com/AlenaResiko/UML/hugging_face/data/huggingface/imdb/test.parquet'
# !wget -O test.parquet "$url"
# df_imdb_test = pd.read_parquet('test.parquet', engine='pyarrow')  # or engine='fastparquet'
# df_imdb_test.head()

--2025-10-18 17:27:10--  https://raw.githubusercontent.com/AlenaResiko/UML/hugging_face/data/huggingface/imdb/train.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33239331 (32M) [application/octet-stream]
Saving to: ‘train.parquet’

train.parquet       100%[===================>]  31.70M   167MB/s    in 0.2s    

2025-10-18 17:27:10 (167 MB/s) - ‘train.parquet’ saved [33239331/33239331]

--2025-10-18 17:27:11--  https://raw.githubusercontent.com/AlenaResiko/UML/hugging_face/data/huggingface/imdb/test.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, aw

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0


In [ ]:
# df_imdb_train.describe()

,label
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


In [ ]:
# df_imdb_train['text'][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

## Sentence-compression

In [ ]:
from datasets import load_dataset, DatasetDict, load_from_disk

dataset_name = "embedding-data/sentence-compression"
dataset = load_dataset(dataset_name)

print("Dataset structure before splitting and transforming:", dataset)

# Assuming the dataset has a 'train' split, split it into train, validation, and test
if 'train' in dataset:
    print("Splitting the 'train' split into train, validation, and test (80/10/10).")
    # Split train into train and test (80/20)
    train_test_split = dataset['train'].train_test_split(test_size=0.2, seed=42)
    train_split = train_test_split['train']
    test_validation_split = train_test_split['test']

    # Split the 20% test set into validation and test (50/50, resulting in 10% validation and 10% test of original)
    validation_test_split = test_validation_split.train_test_split(test_size=0.5, seed=42)
    validation_split = validation_test_split['train']
    test_split = validation_test_split['test']

    # Create a new DatasetDict with the splits
    dataset_splits = DatasetDict({
        'train': train_split,
        'validation': validation_split,
        'test': test_split
    })

    print("\nDataset structure after splitting:", dataset_splits)

    # Define a function to transform each example
    def transform_example(example):
        if isinstance(example['set'], list) and len(example['set']) == 2:
            return {
                'sentence1': example['set'][0],
                'sentence2': example['set'][1],
                'label': 1.0  # Add the label with value 1
            }
        else:
            # Return None or handle cases that don't match the expected format
            return None

    # Apply the transformation to each split
    transformed_dataset_splits = DatasetDict()
    for split_name, split_dataset in dataset_splits.items():
        transformed_dataset_splits[split_name] = split_dataset.map(transform_example, remove_columns=['set'])

    dataset_splits = transformed_dataset_splits

    print("\nDataset structure after transforming:", dataset_splits)
    print("\nNumber of examples in train split:", len(dataset_splits['train']))
    print("Number of examples in validation split:", len(dataset_splits['validation']))
    print("Number of examples in test split:", len(dataset_splits['test']))

    # Save the splits locally
    save_path = './sentence-compression-dataset'
    dataset_splits.save_to_disk(save_path)
    print(f"\nDataset splits saved to {save_path}")

else:
    print("Dataset does not contain a 'train' split. Cannot perform splitting.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sentence-compression_compressed.jsonl.gz:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/180000 [00:00<?, ? examples/s]

Dataset structure before splitting and transforming: DatasetDict({
    train: Dataset({
        features: ['set'],
        num_rows: 180000
    })
})
Splitting the 'train' split into train, validation, and test (80/10/10).

Dataset structure after splitting: DatasetDict({
    train: Dataset({
        features: ['set'],
        num_rows: 144000
    })
    validation: Dataset({
        features: ['set'],
        num_rows: 18000
    })
    test: Dataset({
        features: ['set'],
        num_rows: 18000
    })
})


...


Dataset structure after transforming: DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 144000
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 18000
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 18000
    })
})

Number of examples in train split: 144000
Number of examples in validation split: 18000
Number of examples in test split: 18000


Saving the dataset (0/1 shards):   0%|          | 0/144000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/18000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/18000 [00:00<?, ? examples/s]


Dataset splits saved to ./sentence-compression-dataset


In [ ]:
print("Dataset structure:", dataset_splits)

# Display information about the training split
keys = ['train', 'validation', 'test']
for key in keys:
  if key in dataset_splits:
      print(f"\n{key} split info:")
      print(dataset_splits[key])
      print("\nFeatures:", dataset_splits[key].features)
      print(f"\nNumber of examples in {key} split:", len(dataset_splits[key]))

      # Display the first few examples from the training split
      print("\nFirst 5 examples from the training split:")
      for i in range(min(5, len(dataset_splits[key]))):
          print(f"Example {i}: {dataset_splits[key][i]}")

Dataset structure: DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 144000
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 18000
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 18000
    })
})

train split info:
Dataset({
    features: ['sentence1', 'sentence2', 'label'],
    num_rows: 144000
})

Features: {'sentence1': Value('string'), 'sentence2': Value('string'), 'label': Value('float64')}

Number of examples in train split: 144000

First 5 examples from the training split:
Example 0: {'sentence1': 'A Michigan man has pleaded guilty to persuading mothers in several states to sexually assault their young children and send him images.', 'sentence2': 'Mich. man pleads guilty to persuading mothers to sexually assault their young children', 'label': 1.0}
Example 1: {'sentence1': "Isipathana made it three in a row when the

## Scientific x Taylor

# Model

In [ ]:
# -------------------------
# Model creation helper
# -------------------------
def build_model(base_model_name: str, target_dim: int) -> SentenceTransformer:
    """
    Build a SentenceTransformer where we append a Dense projection after pooling
    to obtain exactly `target_dim` output dimensions.
    """
    # Transformer (encoder)
    word_embedding_model = models.Transformer(base_model_name, max_seq_length=128)

    # Mean pooling (or use cls pooling if you prefer)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,  # sentence embedding = mean of word embeddings in sentence, that's rule of thumb for sentence similarity but if we want to do classification prob cls is better
        pooling_mode_cls_token=False, # instead of cls or max use mean here; ABOBA: can vary and see changes
        pooling_mode_max_tokens=False
    )

    # The pooler (projector)
    dense = models.Dense(
        in_features=pooling_model.get_sentence_embedding_dimension(),
        out_features=target_dim, # ABOBA: vary output dim
        activation_function=nn.Tanh()  # paper used typical pooler activations; Tanh is common
    )

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense], device=DEVICE)
    return model

# -------------------------
# Data loaders (contrastive / STS demo)
# -------------------------
def load_sts_train_eval(): # paper used sts for evaluation
    """
    Load STS-B dataset (train/validation/test) from 'glue' or 'stsb_multi_mt'.
    We use this both for training demo and evaluation (small-scale).
    Replace with large contrastive dataset for better training (e.g., NLI+hard-negatives).
    """
    ds = load_dataset("glue", "stsb") # ABOBA: @alena add Aksel's datasets
    train = ds["train"]
    val = ds["validation"]
    test = ds["test"]

    # Prepare SentenceTransformers InputExample format for regression (score in [0,1])
    def to_examples(split):
        examples = []
        for row in split:
            s1 = row["sentence1"]
            s2 = row["sentence2"]
            score = float(row["label"]) / 5.0  # STS-B scores 0..5 -> normalize to 0..1
            examples.append(InputExample(texts=[s1, s2], label=score))
        return examples

    return to_examples(train), to_examples(val), to_examples(test)

In [ ]:
# -------------------------
# Training functions
# -------------------------
def freeze_encoder_only(model: SentenceTransformer):
    # SentenceTransformer stores modules in model._modules (OrderedDict-like). The transformer is index 0.
    # Simpler: freeze parameters in modules that are instances of models.Transformer
    for module in model._modules.values():
        if isinstance(module, models.Transformer):
            for p in module.parameters():
                p.requires_grad = False

def unfreeze_all(model: SentenceTransformer):
    for p in model.parameters(): # note .parameters() is built-in nn.Module from whihc SentenceTransformer and its submodules inherit
        p.requires_grad = True

def train_pooler_then_finetune(model: SentenceTransformer, train_examples, val_examples, out_dir: str):
    # Step A: train pooler only (encoder frozen)
    freeze_encoder_only(model)
    train_dataloader = torch.utils.data.DataLoader(train_examples, batch_size=BATCH_SIZE, shuffle=True)
    # Use CosineSimilarityLoss for contrastive-style or MSELoss for regression (STS)
    loss_fct = losses.CosineSimilarityLoss(model)  # ABOBA: try dfferent distances; use losses.MultipleNegativesRankingLoss for regression
    # regression objective (STS) -> losses.SoftTargetLoss or losses.SentenceLabelLoss
    # sentence-transformers doesn't provide direct regression loss we can wrap a MSE by computing cosine and matching target score. For simplicity, use CosineSimilarityLoss and treat high score -> similar.
    evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(val_examples, name="sts-val") # note this benchmark compares against human-annotated similarity scores; ABOBA: we can't self-annotate sim for Swift or Verma so we can't get encoder error

    model.fit(
        train_objectives=[(train_dataloader, loss_fct)],
        evaluator=evaluator,
        epochs=EPOCHS_POOLER,
        warmup_steps=100,
        output_path=os.path.join(out_dir, "stepA_pooler_only"),
        optimizer_params={'lr': POOLER_LR}
    )

    # Step B: unfreeze encoder and finetune whole model
    unfreeze_all(model)
    # Recreate dataloader (sentence-transformers expects InputExamples in an in-memory list)
    train_dataloader = torch.utils.data.DataLoader(train_examples, batch_size=BATCH_SIZE, shuffle=True)
    loss_fct2 = losses.MultipleNegativesRankingLoss(model)  # good objective for contrastive training (requires positive pairs)
    model.fit(
        train_objectives=[(train_dataloader, loss_fct2)],
        evaluator=evaluator,
        epochs=EPOCHS_FINETUNE,
        warmup_steps=100,
        output_path=os.path.join(out_dir, "stepB_finetune"),
        optimizer_params={'lr': FINETUNE_LR}
    )

In [ ]:
# -------------------------
# Evaluation helpers
# -------------------------
def evaluate_sts(model: SentenceTransformer, examples):
    """Compute Pearson & Spearman on STS-style examples using sentence-transformers evaluator utilities."""
    evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(examples, name="sts-eval")
    return evaluator(model)

def compute_pca_explained_variance(embeddings: np.ndarray, n_components: int = 50) -> Tuple[np.ndarray, np.ndarray]:
    pca = PCA(n_components=n_components)
    pca.fit(embeddings)
    explained = pca.explained_variance_ratio_
    cum = np.cumsum(explained)
    return explained, cum

def participation_ratio(singular_values: np.ndarray) -> float:
    """
    Participation ratio = (sum_i s_i^2)^2 / sum_i s_i^4
    When s_i are singular values of embedding matrix (or eigenvalues).
    Higher -> more dimensions effectively used.
    """
    s2 = singular_values ** 2
    num = (s2.sum()) ** 2
    den = (s2 ** 2).sum()
    if den == 0:
        return 0.0
    return num / den

In [ ]:
"""
This is reproduction the two-step training idea from:
"On the Dimensionality of Sentence Embeddings" (Wang et al., EMNLP Findings 2023).
that paper didn't include code
"""

import os
from typing import Tuple
import numpy as np
import torch
from torch import nn
from datasets import load_dataset, DatasetDict, load_from_disk # Import load_from_disk
from sentence_transformers import SentenceTransformer, models, losses, InputExample, evaluation
from sentence_transformers.readers import STSBenchmarkDataReader
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import evaluate

# -------------------------
# Config
# -------------------------
BASE_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # ABOBA: small, fast baseline; swap as desired; can use BERT
TARGET_DIM = 32         # ABOBA desired embedding dimensionality (experiment with 32, 64, 128...)
BATCH_SIZE = 64
POOLER_LR = 2e-4
FINETUNE_LR = 2e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCHS_POOLER = 2      # step A epochs (pooler only)
EPOCHS_FINETUNE = 2    # step B epochs (unfreeze and train)
SAVE_DIR = "./lowdim_model"
CUSTOM_DATASET_PATH = "./sentence-compression-dataset"
os.makedirs(SAVE_DIR, exist_ok=True)

# -------------------------
# Model creation helper
# -------------------------
def build_model(base_model_name: str, target_dim: int) -> SentenceTransformer:
    """
    Build a SentenceTransformer where we append a Dense projection after pooling
    to obtain exactly `target_dim` output dimensions.
    """
    # Transformer (encoder)
    word_embedding_model = models.Transformer(base_model_name, max_seq_length=128)

    # Mean pooling (or use cls pooling if you prefer)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,  # sentence embedding = mean of word embeddings in sentence, that's rule of thumb for sentence similarity but if we want to do classification prob cls is better
        pooling_mode_cls_token=False, # instead of cls or max use mean here; ABOBA: can vary and see changes
        pooling_mode_max_tokens=False
    )

    # The pooler (projector)
    dense = models.Dense(
        in_features=pooling_model.get_sentence_embedding_dimension(),
        out_features=target_dim, # ABOBA: vary output dim
        activation_function=nn.Tanh()  # paper used typical pooler activations; Tanh is common
    )

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense], device=DEVICE)
    return model

# -------------------------
# Data loaders (contrastive / STS demo)
# -------------------------
# def load_sts_train_eval(): # paper used sts for evaluation
#     """
#     Load STS-B dataset (train/validation/test) from 'glue' or 'stsb_multi_mt'.
#     We use this both for training demo and evaluation (small-scale).
#     Replace with large contrastive dataset for better training (e.g., NLI+hard-negatives).
#     """
#     ds = load_dataset("glue", "stsb") # ABOBA: @alena add Aksel's datasets
#     train = ds["train"]
#     val = ds["validation"]
#     test = ds["test"]

#     # Prepare SentenceTransformers InputExample format for regression (score in [0,1])
#     def to_examples(split):
#         examples = []
#         for row in split:
#             s1 = row["sentence1"]
#             s2 = row["sentence2"]
#             score = float(row["label"]) / 5.0  # STS-B scores 0..5 -> normalize to 0..1
#             examples.append(InputExample(texts=[s1, s2], label=score))
#         return examples

#     return to_examples(train), to_examples(val), to_examples(test)

def load_custom_dataset(dataset_path: str):
    """
    Load a custom dataset from a local path or Hugging Face Hub.
    Assumes the dataset is saved in a format loadable by datasets (e.g., parquet, json, csv).
    Assumes the dataset has 'sentence1', 'sentence2', and 'label' columns.
    Labels are assumed to be similarity scores.
    If only a 'train' split is available, it will be split into train, validation, and test sets.
    """
    ds = None
    if dataset_path and os.path.exists(dataset_path):
        try:
            # Try loading from a directory (e.g., saved with dataset.save_to_disk)
            ds = load_from_disk(dataset_path)
            print(f"Loaded dataset from disk: {dataset_path}")
        except Exception as e_disk:
            print(f"Could not load dataset from disk: {e_disk}")
            try:
                # Try loading from common file formats (csv, json, parquet)
                if dataset_path.endswith('.csv'):
                     ds = load_dataset('csv', data_files=dataset_path)
                elif dataset_path.endswith('.json'):
                     ds = load_dataset('json', data_files=dataset_path)
                elif dataset_path.endswith('.parquet'):
                     ds = load_dataset('parquet', data_files=dataset_path)
                else:
                     print(f"Unsupported file format for custom dataset: {dataset_path}")
                     return None
                print(f"Loaded dataset from file: {dataset_path}")
            except Exception as e_file:
                print(f"Could not load dataset from file: {e_file}")
                return None
    elif dataset_path:
        # Try loading from Hugging Face Hub if dataset_path is not a local path
        try:
            ds = load_dataset(dataset_path)
            print(f"Loaded dataset from Hugging Face Hub: {dataset_path}")
        except Exception as e_hub:
            print(f"Could not load dataset from Hugging Face Hub: {e_hub}")
            return None
    else:
        print("No dataset path provided.")
        return None


    if ds is None:
        return None

    # Ensure ds is a DatasetDict for consistent handling
    if not isinstance(ds, DatasetDict):
        if 'train' in ds.features: # Check if it's a single split Dataset
             ds = DatasetDict({'train': ds})
             print("Wrapped single dataset split in a DatasetDict.")
        else:
            print("Loaded dataset is not in a recognized format (Dataset or DatasetDict with 'train' split).")
            return None


    def to_examples(split):
        examples = []
        # Check if required columns exist
        if "sentence1" not in split.column_names or "sentence2" not in split.column_names or "label" not in split.column_names:
            print("Custom dataset must contain 'sentence1', 'sentence2', and 'label' columns.")
            return []
        for row in split:
            s1 = row["sentence1"]
            s2 = row["sentence2"]
            score = float(row["label"]) # Assume label is already a float similarity score
            examples.append(InputExample(texts=[s1, s2], label=score))
        return examples

    custom_data = {}
    if "train" in ds:
        # If only train split, split it
        if "validation" not in ds and "test" not in ds:
            print("Splitting the 'train' split into train, validation, and test (80/10/10).")
            ds_split = ds["train"].train_test_split(test_size=0.2, seed=42)
            ds_validation_test = ds_split["test"].train_test_split(test_size=0.5, seed=42)
            custom_data["train"] = to_examples(ds_split["train"])
            custom_data["validation"] = to_examples(ds_validation_test["train"])
            custom_data["test"] = to_examples(ds_validation_test["test"])
        else:
            custom_data["train"] = to_examples(ds["train"])
            if "validation" in ds:
                 custom_data["validation"] = to_examples(ds["validation"])
            if "test" in ds:
                 custom_data["test"] = to_examples(ds["test"])


    return custom_data


# -------------------------
# Training functions
# -------------------------
def freeze_encoder_only(model: SentenceTransformer):
    # SentenceTransformer stores modules in model._modules (OrderedDict-like). The transformer is index 0.
    # Simpler: freeze parameters in modules that are instances of models.Transformer
    for module in model._modules.values():
        if isinstance(module, models.Transformer):
            for p in module.parameters():
                p.requires_grad = False

def unfreeze_all(model: SentenceTransformer):
    for p in model.parameters(): # note .parameters() is built-in nn.Module from whihc SentenceTransformer and its submodules inherit
        p.requires_grad = True

def train_pooler_then_finetune(model: SentenceTransformer, train_examples, val_examples, out_dir: str):
    # Step A: train pooler only (encoder frozen)
    freeze_encoder_only(model)
    train_dataloader = torch.utils.data.DataLoader(train_examples, batch_size=BATCH_SIZE, shuffle=True)
    # Use CosineSimilarityLoss for contrastive-style or MSELoss for regression (STS)
    loss_fct = losses.CosineSimilarityLoss(model)  # ABOBA: try dfferent distances; use losses.MultipleNegativesRankingLoss for regression
    # regression objective (STS) -> losses.SoftTargetLoss or losses.SentenceLabelLoss
    # sentence-transformers doesn't provide direct regression loss we can wrap a MSE by computing cosine and matching target score. For simplicity, use CosineSimilarityLoss and treat high score -> similar.
    evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(val_examples, name="sts-val") # note this benchmark compares against human-annotated similarity scores; ABOBA: we can't self-annotate sim for Swift or Verma so we can't get encoder error

    model.fit(
        train_objectives=[(train_dataloader, loss_fct)],
        evaluator=evaluator,
        epochs=EPOCHS_POOLER,
        warmup_steps=100,
        output_path=os.path.join(out_dir, "stepA_pooler_only"),
        optimizer_params={'lr': POOLER_LR}
    )

    # Step B: unfreeze encoder and finetune whole model
    unfreeze_all(model)
    # Recreate dataloader (sentence-transformers expects InputExamples in an in-memory list)
    train_dataloader = torch.utils.data.DataLoader(train_examples, batch_size=BATCH_SIZE, shuffle=True)
    loss_fct2 = losses.MultipleNegativesRankingLoss(model)  # good objective for contrastive training (requires positive pairs)
    model.fit(
        train_objectives=[(train_dataloader, loss_fct2)],
        evaluator=evaluator,
        epochs=EPOCHS_FINETUNE,
        warmup_steps=100,
        output_path=os.path.join(out_dir, "stepB_finetune"),
        optimizer_params={'lr': FINETUNE_LR}
    )

# -------------------------
# Evaluation helpers
# -------------------------
def evaluate_sts(model: SentenceTransformer, examples):
    """Compute Pearson & Spearman on STS-style examples using sentence-transformers evaluator utilities."""
    evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(examples, name="sts-eval")
    return evaluator(model)

def compute_pca_explained_variance(embeddings: np.ndarray, n_components: int = 50) -> Tuple[np.ndarray, np.ndarray]:
    pca = PCA(n_components=n_components)
    pca.fit(embeddings)
    explained = pca.explained_variance_ratio_
    cum = np.cumsum(explained)
    return explained, cum

def participation_ratio(singular_values: np.ndarray) -> float:
    """
    Participation ratio = (sum_i s_i^2)^2 / sum_i s_i^4
    When s_i are singular values of embedding matrix (or eigenvalues).
    Higher -> more dimensions effectively used.
    """
    s2 = singular_values ** 2
    num = (s2.sum()) ** 2
    den = (s2 ** 2).sum()
    if den == 0:
        return 0.0
    return num / den

# -------------------------
# Main: build, train, evaluate
# -------------------------
def main():
    print("Device:", DEVICE)
    # 1) Build model with custom pooler/projection to TARGET_DIM
    model = build_model(BASE_MODEL, TARGET_DIM)
    print("Model built. Output dim:", model.get_sentence_embedding_dimension())

    train_examples = []
    val_examples = []
    test_examples = []

    # Load custom dataset if specified
    custom_data = load_custom_dataset(CUSTOM_DATASET_PATH)
    if custom_data:
        print(f"Loaded custom dataset: train={len(custom_data.get('train', []))} val={len(custom_data.get('validation', []))} test={len(custom_data.get('test', []))}")
        # Use custom data for training, validation, and testing if available
        train_examples = custom_data.get("train", [])
        val_examples = custom_data.get("validation", [])
        test_examples = custom_data.get("test", [])
        print("Using custom dataset for training, validation, and evaluation.")
    else:
        # 2) Load STS train/eval (demo) if no custom dataset is provided
        train_examples, val_examples, test_examples = load_sts_train_eval()
        print(f"Loaded STS: train={len(train_examples)} val={len(val_examples)} test={len(test_examples)}")
        print("Using STS-B dataset for training, validation, and evaluation.")


    if not train_examples or not val_examples or not test_examples:
        print("Error: Training, validation, and/or test datasets are empty or could not be loaded. Cannot proceed with training and evaluation.")
        return

    # 3) Train: two-step
    train_pooler_then_finetune(model, train_examples, val_examples, SAVE_DIR)

    # 4) Evaluate on test
    print("Evaluating on test set...")
    evaluate_sts(model, test_examples)

    # 5) Dump embeddings for intrinsic-dim analysis (take a subset for speed)
    sentences = [ex.texts[0] for ex in test_examples][:1000]  # first sentence of pairs, subset
    embeddings = model.encode(sentences, show_progress_bar=True, convert_to_numpy=True)
    print("Embeddings shape:", embeddings.shape)

    # PCA info
    explained, cum = compute_pca_explained_variance(embeddings, n_components=min(embeddings.shape[1], 50))
    print("Explained variance ratios (first 10):", explained[:10])
    print("Cumulative variance (first 10):", cum[:10])

    # Participation ratio (use SVD of covariance)
    cov = np.cov(embeddings, rowvar=False)
    eigvals = np.linalg.eigvalsh(cov)  # ascending
    eigvals = eigvals[eigvals > 0]
    pr = participation_ratio(np.sqrt(eigvals))  # pass singular values ~ sqrt(eigvals)
    print(f"Participation ratio (approx): {pr:.3f}")

    # Save final model
    model.save(os.path.join(SAVE_DIR, "final_lowdim_model"))
    print("Saved model to", os.path.join(SAVE_DIR, "final_lowdim_model"))


if __name__ == "__main__":
    main()

Device: cuda


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alenachan121 (alenachan121-columbia-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/sentence_transformers/evaluation/EmbeddingSimilarityEvaluator.py:195: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  eval_pearson, _ = pearsonr(labels, scores)
/usr/local/lib/python3.12/dist-packages/sentence_transformers/evaluation/EmbeddingSimilarityEvaluator.py:196: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  eval_spearman, _ = spearmanr(labels, scores)


Step,Training Loss


KeyboardInterrupt: 

# Use contrastive learning

In [ ]:
import random
from typing import List, Tuple, Optional
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer
from tqdm.auto import tqdm
import numpy as np
from scipy.stats import spearmanr

# ------------------------
# Utilities / Dataset
# ------------------------

class SentenceCorpus:
    """
    Holder for a corpus of sentences with their document and position info.
    Input: list of documents, where each document is list[str] (sentences in order).
    This lets us define 'neighbor' sentences (within a distance window).
    """
    def __init__(self, docs: List[List[str]]):
        self.docs = docs
        # flatten to a list of (doc_id, sent_idx, text)
        self.flat = []
        for d_id, doc in enumerate(docs):
            for s_idx, text in enumerate(doc):
                self.flat.append((d_id, s_idx, text))
        self.N = len(self.flat)

    def get_sentence(self, flat_idx: int) -> str:
        return self.flat[flat_idx][2]

    def neighbor_indices(self, flat_idx: int, max_dist: int = 10) -> List[int]:
        """Return indices of sentences within distance < max_dist in same document (excluding itself)."""
        d_id, s_idx, _ = self.flat[flat_idx]
        doc = self.docs[d_id]
        neighbors = []
        lo = max(0, s_idx - max_dist + 1)
        hi = min(len(doc) - 1, s_idx + max_dist - 1)
        if lo <= hi:
            for j in range(lo, hi + 1):
                if j == s_idx:
                    continue
                # map (d_id, j) back to flat index: we can scan (cheap once) or build index map
                # Let's build mapping once:
            # we'll use a mapping built in constructor
        return []  # unused; dataset uses precomputed mapping

class ContrastiveSentenceDataset(Dataset):
    """
    Dataset returning an anchor index. The collate function constructs multiple text inputs:
      - anchor_text (used twice for dropout augmentations in model forward)
      - neighbor_texts (0..k positives from nearby sentences)
    The collator will produce tokenized batches.
    """
    def __init__(self,
                 docs: List[List[str]],
                 neighbor_window: int = 10,
                 num_neighbors: int = 1,
                 sample_neighbors_prob: float = 1.0):
        """
        docs: list of documents (each is list of sentences)
        neighbor_window: maximum sentence distance to consider neighbor (distance < neighbor_window)
        num_neighbors: number of neighbor positives to sample per anchor (if available)
        sample_neighbors_prob: probability to include neighbor positives (for mixing supervised/unsupervised)
        """
        self.docs = docs
        self.corpus = SentenceCorpus(docs)
        self.num_neighbors = num_neighbors
        self.neighbor_window = neighbor_window
        self.sample_neighbors_prob = sample_neighbors_prob

        # build mapping (doc_id, sent_idx) -> flat index for quick neighbor lookup
        self.doc_index_starts = []
        flat_idx = 0
        for doc in docs:
            self.doc_index_starts.append(flat_idx)
            flat_idx += len(doc)

        # map (doc_id, sent_idx) -> flat_index:
        self.doc_pos_to_flat = {}
        flat = 0
        for d_id, doc in enumerate(docs):
            for s_idx, _ in enumerate(doc):
                self.doc_pos_to_flat[(d_id, s_idx)] = flat
                flat += 1
        self.N = flat

    def __len__(self):
        return self.N

    def sample_neighbors_for_flat_idx(self, flat_idx: int) -> List[int]:
        d_id, s_idx, _ = self.corpus.flat[flat_idx]
        doc = self.docs[d_id]
        lo = max(0, s_idx - (self.neighbor_window - 1))
        hi = min(len(doc) - 1, s_idx + (self.neighbor_window - 1))
        candidates = [j for j in range(lo, hi + 1) if j != s_idx]
        if not candidates:
            return []
        k = min(self.num_neighbors, len(candidates))
        sampled = random.sample(candidates, k)
        flat_sampled = [self.doc_pos_to_flat[(d_id, j)] for j in sampled]
        return flat_sampled

    def __getitem__(self, idx: int):
        """
        Returns an item describing the anchor and indices of positives:
          {
            'anchor_idx': int,
            'anchor_text': str,
            'neighbor_pos_indices': List[int]  # could be empty
          }
        """
        anchor_text = self.corpus.get_sentence(idx)
        neighbor_indices = []
        if random.random() < self.sample_neighbors_prob:
            neighbor_indices = self.sample_neighbors_for_flat_idx(idx)
        return {
            'anchor_idx': idx,
            'anchor_text': anchor_text,
            'neighbor_pos_indices': neighbor_indices
        }

def collate_contrastive(batch_items: List[dict], tokenizer, max_length=64):
    """
    We will produce tokenized inputs for:
      - anchors (we will compute embeddings twice with dropout for SimCSE-style augment)
      - neighbor positives
    Return structure with:
      - input_texts: List[str]
      - metadata: per anchor offsets to locate where anchor copies & neighbors are
      - tokenized inputs as tensors (input_ids, attention_mask)
    """
    input_texts = []
    metadata = []
    for it in batch_items:
        a = it['anchor_text']
        # anchor text will appear twice (two dropout views)
        anchor_pos = len(input_texts)
        input_texts.append(a)
        anchor_pos2 = len(input_texts)
        input_texts.append(a)
        neighbor_positions = []
        for n_idx in it['neighbor_pos_indices']:
            neigh_text = tokenizer.decode(tokenizer.encode(corpus.flat[n_idx][2], max_length=max_length, truncation=True))
            # BUT decoding/encoding is inefficient — instead, we'll access text directly:
            neighbor_positions.append(len(input_texts))
            input_texts.append(corpus.get_sentence(n_idx))
        metadata.append({
            'anchor_idx': it['anchor_idx'],
            'anchor_pos': anchor_pos,
            'anchor_pos2': anchor_pos2,
            'neighbor_positions': neighbor_positions
        })
    # Tokenize all texts at once
    tok = tokenizer(input_texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    return tok, metadata

# ------------------------
# Model
# ------------------------

class SentenceEmbedder(nn.Module):
    """
    BERT-based sentence embedder with mean-pooling or cls pooling.
    The model is built so passing the same input twice with dropout yields different embeddings
    (SimCSE unsupervised trick).
    """
    def __init__(self, model_name='bert-base-uncased', pooling: str = 'mean', device='cuda'):
        super().__init__()
        assert pooling in ('mean', 'cls')
        self.device = device
        self.enc = AutoModel.from_pretrained(model_name)
        self.pooling = pooling
        # optionally add a projection head (useful for contrastive learning)
        hidden_size = self.enc.config.hidden_size
        proj_size = hidden_size  # can change to smaller dim
        self.projection = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, proj_size)
        )
        self.to(device)

    def forward(self, input_ids, attention_mask):
        """
        Encode batch of tokenized sentences and return normalized embeddings.
        input_ids, attention_mask: tensors (batch, seq_len)
        Returns: L2-normalized embeddings (batch, proj_size)
        """
        outputs = self.enc(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        last_hidden = outputs.last_hidden_state  # (B, L, H)
        if self.pooling == 'mean':
            # mean pooling over tokens with attention mask
            mask = attention_mask.unsqueeze(-1).type_as(last_hidden)  # (B, L, 1)
            summed = (last_hidden * mask).sum(1)  # (B, H)
            denom = mask.sum(1).clamp(min=1e-9)
            pooled = summed / denom
        else:
            pooled = outputs.pooler_output if outputs.pooler_output is not None else last_hidden[:,0]
        z = self.projection(pooled)
        z = F.normalize(z, p=2, dim=1)
        return z

# ------------------------
# Loss: multi-positive NT-Xent
# ------------------------

def multi_positive_nt_xent(embeddings: torch.FloatTensor,
                            metadata: List[dict],
                            temperature: float = 0.05,
                            device='cuda'):
    """
    embeddings: (M, D) where M is number of texts in the batch tokenization (including anchor copies and neighbors)
    metadata: list per anchor with positions where anchor copies and neighbor positions are located inside embeddings
    We will treat for each anchor:
      anchor_pos is index i, positives are anchor_pos2 and any neighbor positions.
    Compute loss: -log( sum_{p in P(i)} exp(sim(i,p)/temp) / sum_{k != i} exp(sim(i,k)/temp) )
    and average over anchors. Also consider the symmetric direction? We'll average over anchors only (anchors are half of examples).
    """
    device = embeddings.device
    sim = torch.matmul(embeddings, embeddings.T)  # (M, M) cosine because embeddings are normalized
    sim = sim / temperature

    M = sim.shape[0]
    losses = []
    large_neg_mask = torch.eye(M, dtype=torch.bool, device=device)  # remove self-sim
    exp_sim = torch.exp(sim)
    for m, meta in enumerate(metadata):
        i = meta['anchor_pos']
        positives = [meta['anchor_pos2']] + meta['neighbor_positions']  # positions of positives
        if len(positives) == 0:
            # fallback: treat anchor_pos2 only (should always exist)
            positives = [meta['anchor_pos2']]
        # numerator: sum over positives
        numer = exp_sim[i, positives].sum()
        # denominator: sum over all except i itself
        denom = exp_sim[i].masked_fill(large_neg_mask[i], 0.0).sum()
        # safety
        denom = denom.clamp(min=1e-9)
        loss_i = -torch.log(numer / denom)
        losses.append(loss_i)
    loss = torch.stack(losses).mean()
    return loss

# ------------------------
# Training loop
# ------------------------

def train_contrastive(model: SentenceEmbedder,
                      tokenizer,
                      dataset: ContrastiveSentenceDataset,
                      corpus_obj: SentenceCorpus,
                      device='cuda',
                      epochs: int = 1,
                      batch_size: int = 64,
                      lr: float = 2e-5,
                      max_length: int = 64,
                      temperature: float = 0.05,
                      eval_every: Optional[int] = None,
                      num_workers: int = 2):
    """
    Training driver. Uses the dataset's __getitem__ and a custom collate function.
    NOTE: this implementation uses the tokenizer outside dataset, and constructs
    the tokenized batch from text. For efficiency you may want to pre-tokenize corpus.
    """
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=lambda x: collate_batch_fast(x, tokenizer, max_length, corpus_obj))
    pbar = tqdm(range(epochs * len(loader)), desc="Train steps")
    step = 0
    for epoch in range(epochs):
        for batch_tok, metadata in loader:
            # batch_tok: dict of tensors
            # metadata: list per anchor
            input_ids = batch_tok['input_ids'].to(device)
            attention_mask = batch_tok['attention_mask'].to(device)
            # forward once: because anchors repeated in token list, embeddings will correspond
            embeddings = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = multi_positive_nt_xent(embeddings, metadata, temperature=temperature, device=device)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pbar.set_postfix({'loss': loss.item()})
            pbar.update(1)
            step += 1
        # epoch end: (optionally evaluate)
    pbar.close()
    return model

# Helper: faster collate that avoids repeated tokenization decode/encode
def collate_batch_fast(batch_items: List[dict], tokenizer, max_length: int, corpus: SentenceCorpus):
    """
    Build list of texts more efficiently without decode/encode roundtrip.
    """
    texts = []
    metadata = []
    for it in batch_items:
        a = it['anchor_text']
        anchor_pos = len(texts)
        texts.append(a)        # view 1
        anchor_pos2 = len(texts)
        texts.append(a)        # view 2
        neighbor_positions = []
        for flat_idx in it['neighbor_pos_indices']:
            neighbor_positions.append(len(texts))
            texts.append(corpus.get_sentence(flat_idx))
        metadata.append({
            'anchor_idx': it['anchor_idx'],
            'anchor_pos': anchor_pos,
            'anchor_pos2': anchor_pos2,
            'neighbor_positions': neighbor_positions
        })
    tok = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    return tok, metadata

# ------------------------
# Evaluation utilities
# ------------------------

def embed_sentences(model: SentenceEmbedder, tokenizer, texts: List[str], device='cuda', batch_size=128, max_length=64):
    model.eval()
    all_embs = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            tok = tokenizer(batch_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt').to(device)
            embs = model(tok['input_ids'], tok['attention_mask'])
            all_embs.append(embs.cpu().numpy())
    return np.vstack(all_embs)

def evaluate_similarity_pairs(model: SentenceEmbedder, tokenizer, pairs: List[Tuple[str, str]], gold_scores: Optional[List[float]] = None):
    """
    Compute cosine similarity for pairs. If gold_scores provided, compute Spearman correlation.
    """
    texts = []
    for a, b in pairs:
        texts.append(a)
        texts.append(b)
    embs = embed_sentences(model, tokenizer, texts)
    sims = []
    for i in range(0, len(texts), 2):
        a_emb = embs[i]
        b_emb = embs[i+1]
        sim = float(np.dot(a_emb, b_emb) / (np.linalg.norm(a_emb) * np.linalg.norm(b_emb) + 1e-9))
        sims.append(sim)
    if gold_scores is not None:
        rho, p = spearmanr(sims, gold_scores)
        return sims, rho, p
    return sims, None, None

# ------------------------
# Example usage / Putting it together
# ------------------------

if __name__ == '__main__':
    # Example minimal usage (use your own corpus)
    model_name = 'bert-base-uncased'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Dummy toy corpus: list of documents, each is list of sentences
    docs = [
        ["This is the first sentence.", "Here is a nearby sentence.", "Another sentence in the same doc."],
        ["Unrelated document starts here.", "More text in other doc.", "Even more text."]
    ]
    corpus = SentenceCorpus(docs)
    dataset = ContrastiveSentenceDataset(docs, neighbor_window=3, num_neighbors=1, sample_neighbors_prob=0.9)

    model = SentenceEmbedder(model_name=model_name, pooling='mean', device=device)

    # Train for 1 epoch on toy data (in real setup use larger batches, many epochs)
    model = train_contrastive(model, tokenizer, dataset, corpus, device=device, epochs=1, batch_size=8, lr=2e-5, max_length=64, temperature=0.05)

    # Example evaluation: build pairs (identical, neighbor, random)
    pairs = []
    gold = []  # 1 = similar, 0 = dissimilar (for simple ROC/accuracy), or continuous scores for Spearman
    # identical
    pairs.append((docs[0][0], docs[0][0])); gold.append(1.0)
    # neighbor (distance < window)
    pairs.append((docs[0][0], docs[0][1])); gold.append(1.0)
    # unrelated
    pairs.append((docs[0][0], docs[1][0])); gold.append(0.0)

    sims, rho, p = evaluate_similarity_pairs(model, tokenizer, pairs, gold_scores=gold)
    print("pair similarities:", sims)
    if rho is not None:
        print("Spearman rho:", rho, "p:", p)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Train steps:   0%|          | 0/1 [00:00<?, ?it/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

# Nov 8

In [ ]:
# Assuming the parquet file is in the content directory
parquet_file_path = '/content/papers.parquet'

try:
    df_papers = pd.read_parquet(parquet_file_path, engine='pyarrow') # or engine='fastparquet'
    print("Parquet file loaded successfully!")
    display(df_papers.head())
except FileNotFoundError:
    print(f"Error: The file {parquet_file_path} was not found.")
except Exception as e:
    print(f"An error occurred while loading the parquet file: {e}")

In [ ]:
import random
from typing import List, Tuple, Optional
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer
from tqdm.auto import tqdm
import numpy as np
from scipy.stats import spearmanr
import pandas as pd

# ------------------------
# Utilities / Dataset
# ------------------------

class SentenceCorpus:
    """
    Holder for a corpus of sentences with their document and position info.
    Input: pandas DataFrame with 'doc_id', 'sent_index', and 'sentence' columns.
    This lets us define 'neighbor' sentences (within a distance window) within each document.
    """
    def __init__(self, df: pd.DataFrame, doc_id_col: str = 'doc_id', sent_index_col: str = 'sent_index', sentence_col: str = 'sentence'):
        self.df = df.sort_values(by=[doc_id_col, sent_index_col]).reset_index(drop=True) # Ensure data is sorted
        self.doc_id_col = doc_id_col
        self.sent_index_col = sent_index_col
        self.sentence_col = sentence_col

        # Group by document to get sentences per document
        self.docs = self.df.groupby(doc_id_col)[sentence_col].apply(list).tolist()

        # Create a flattened list for easy indexing by flat index
        self.flat = []
        for index, row in self.df.iterrows():
            self.flat.append((row[doc_id_col], row[sent_index_col], row[sentence_col]))

        self.N = len(self.flat) # Total number of sentences

        # Build mapping (doc_id, sent_idx) -> flat index for quick neighbor lookup
        self.doc_pos_to_flat = {}
        for flat_idx, (d_id, s_idx, _) in enumerate(self.flat):
             self.doc_pos_to_flat[(d_id, s_idx)] = flat_idx

        # Store mapping from flat index to document index within self.docs
        self.flat_to_doc_info = []
        doc_start_flat_idx = 0
        for doc_idx, doc_sentences in enumerate(self.docs):
             for sent_idx_in_doc in range(len(doc_sentences)):
                 # Assuming df is sorted by doc_id and sent_index,
                 # the flat index corresponds to the sequential order in the sorted df.
                 # The sent_index_in_doc is the 0-based index within the list of sentences for that doc.
                 # The original sent_index from the dataframe might not be 0-based or contiguous within a doc.
                 # Let's use the index within the 'docs' list for s_idx_in_doc and the actual dataframe sent_index for lookup.
                 # A simpler way is to use the flat index to directly get the doc_id and original sent_index from self.flat
                 original_doc_id = self.flat[doc_start_flat_idx + sent_idx_in_doc][0]
                 original_sent_index = self.flat[doc_start_flat_idx + sent_idx_in_doc][1]
                 self.flat_to_doc_info.append({
                     'doc_id': original_doc_id,
                     'sent_index_in_doc': sent_idx_in_doc # 0-based index within the grouped list
                 })
             doc_start_flat_idx += len(doc_sentences)


    def get_sentence(self, flat_idx: int) -> str:
        """Get sentence text by flat index."""
        if 0 <= flat_idx < self.N:
            return self.flat[flat_idx][2]
        return None

    def get_sentence_by_doc_pos(self, doc_id: str, sent_index_in_doc: int) -> str:
        """Get sentence text by document ID and 0-based index within that document."""
        # Find the flat index for this doc_id and sent_index_in_doc
        for flat_idx, info in enumerate(self.flat_to_doc_info):
             if info['doc_id'] == doc_id and info['sent_index_in_doc'] == sent_index_in_doc:
                 return self.flat[flat_idx][2]
        return None


    def neighbor_indices_flat(self, flat_idx: int, max_dist: int = 10) -> List[int]:
        """
        Return flat indices of sentences within distance < max_dist in the same document (excluding itself).
        Uses the precomputed flat index to doc info.
        """
        if not (0 <= flat_idx < self.N):
             return []

        anchor_info = self.flat_to_doc_info[flat_idx]
        anchor_doc_id = anchor_info['doc_id']
        anchor_sent_index_in_doc = anchor_info['sent_index_in_doc']

        neighbors_flat_indices = []
        # Iterate through all sentences, find those in the same doc within distance
        for current_flat_idx in range(self.N):
             current_info = self.flat_to_doc_info[current_flat_idx]
             if current_info['doc_id'] == anchor_doc_id and current_flat_idx != flat_idx:
                 distance = abs(current_info['sent_index_in_doc'] - anchor_sent_index_in_doc)
                 if distance < max_dist:
                     neighbors_flat_indices.append(current_flat_idx)

        return neighbors_flat_indices


class ContrastiveSentenceDataset(Dataset):
    """
    Dataset returning an anchor index. The collate function constructs multiple text inputs:
      - anchor_text (used twice for dropout augmentations in model forward)
      - neighbor_texts (0..k positives from nearby sentences)
    The collator will produce tokenized batches.
    Uses a SentenceCorpus object internally.
    """
    def __init__(self,
                 corpus: SentenceCorpus, # Accept a SentenceCorpus object
                 neighbor_window: int = 10,
                 num_neighbors: int = 1,
                 sample_neighbors_prob: float = 1.0):
        """
        corpus: A SentenceCorpus object containing the data.
        neighbor_window: maximum sentence distance to consider neighbor (distance < neighbor_window)
        num_neighbors: number of neighbor positives to sample per anchor (if available)
        sample_neighbors_prob: probability to include neighbor positives (for mixing supervised/unsupervised)
        """
        self.corpus = corpus
        self.num_neighbors = num_neighbors
        self.neighbor_window = neighbor_window
        self.sample_neighbors_prob = sample_neighbors_prob
        self.N = self.corpus.N # Total number of sentences

    def __len__(self):
        return self.N

    def sample_neighbors_for_flat_idx(self, flat_idx: int) -> List[int]:
        """Sample random neighbors from the corpus for a given flat index."""
        # Use the neighbor_indices_flat method from the corpus
        all_candidates_flat_indices = self.corpus.neighbor_indices_flat(flat_idx, max_dist=self.neighbor_window)

        if not all_candidates_flat_indices:
            return []

        k = min(self.num_neighbors, len(all_candidates_flat_indices))
        sampled_flat_indices = random.sample(all_candidates_flat_indices, k)
        return sampled_flat_indices


    def __getitem__(self, idx: int):
        """
        Returns an item describing the anchor and indices of positives:
          {
            'anchor_idx': int, # Flat index of the anchor sentence
            'anchor_text': str,
            'neighbor_pos_indices': List[int]  # List of flat indices of neighbor positives
          }
        """
        anchor_text = self.corpus.get_sentence(idx)
        neighbor_indices = []
        if random.random() < self.sample_neighbors_prob:
            neighbor_indices = self.sample_neighbors_for_flat_idx(idx)
        return {
            'anchor_idx': idx,
            'anchor_text': anchor_text,
            'neighbor_pos_indices': neighbor_indices
        }

def collate_contrastive(batch_items: List[dict], tokenizer, max_length=64):
    """
    We will produce tokenized inputs for:
      - anchors (we will compute embeddings twice with dropout augmentations in model forward)
      - neighbor positives
    Return structure with:
      - input_texts: List[str]
      - metadata: per anchor offsets to locate where anchor copies & neighbors are
      - tokenized inputs as tensors (input_ids, attention_mask)
    """
    # This collate function is not used directly by train_contrastive, collate_batch_fast is used.
    # Keeping this for reference, but it might be redundant.
    print("Warning: collate_contrastive is called, but collate_batch_fast should be used by the DataLoader.")
    input_texts = []
    metadata = []
    for it in batch_items:
        a = it['anchor_text']
        # anchor text will appear twice (two dropout views)
        anchor_pos = len(input_texts)
        input_texts.append(a)
        anchor_pos2 = len(input_texts)
        input_texts.append(a)
        neighbor_positions = []
        # Note: This original collate_contrastive doesn't have access to the corpus
        # to get neighbor texts from indices. The collate_batch_fast below does.
        # This function is likely unused now that collate_batch_fast is used.
        metadata.append({
            'anchor_idx': it['anchor_idx'],
            'anchor_pos': anchor_pos,
            'anchor_pos2': anchor_pos2,
            'neighbor_positions': it['neighbor_pos_indices'] # Store indices, text will be fetched in collate_batch_fast
        })
    tok = tokenizer(input_texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    return tok, metadata


# ------------------------
# Model
# ------------------------

class SentenceEmbedder(nn.Module):
    """
    BERT-based sentence embedder with mean-pooling or cls pooling.
    The model is built so passing the same input twice with dropout yields different embeddings
    (SimCSE unsupervised trick).
    Includes an optional projection head.
    """
    def __init__(self, model_name='bert-base-uncased', pooling: str = 'mean', device='cuda', proj_size: Optional[int] = None):
        super().__init__()
        assert pooling in ('mean', 'cls')
        self.device = device
        self.enc = AutoModel.from_pretrained(model_name)
        self.pooling = pooling
        hidden_size = self.enc.config.hidden_size

        # Projection head
        if proj_size is None or proj_size == hidden_size:
             # No separate projection if proj_size is None or matches hidden_size
             self.projection = nn.Identity()
             self.proj_size = hidden_size
        else:
            self.projection = nn.Sequential(
                nn.Linear(hidden_size, hidden_size), # Optional intermediate layer
                nn.Tanh(), # Or ReLU or other activation
                nn.Linear(hidden_size, proj_size)
            )
            self.proj_size = proj_size

        self.to(device)

    def forward(self, input_ids, attention_mask):
        """
        Encode batch of tokenized sentences and return normalized embeddings.
        input_ids, attention_mask: tensors (batch, seq_len)
        Returns: L2-normalized embeddings (batch, proj_size)
        """
        # Enable dropout during training for SimCSE-style augmentation
        outputs = self.enc(input_ids=input_ids, attention_mask=attention_mask, return_dict=True, output_hidden_states=True)
        last_hidden = outputs.last_hidden_state # (B, L, H)

        if self.pooling == 'mean':
            # mean pooling over tokens with attention mask
            mask = attention_mask.unsqueeze(-1).type_as(last_hidden) # (B, L, 1)
            summed = (last_hidden * mask).sum(1) # (B, H)
            denom = mask.sum(1).clamp(min=1e-9)
            pooled = summed / denom
        elif self.pooling == 'cls':
            # Use the CLS token embedding
            pooled = last_hidden[:, 0]
        else:
             raise ValueError(f"Unknown pooling mode: {self.pooling}")

        z = self.projection(pooled)
        z = F.normalize(z, p=2, dim=1)
        return z

    def get_sentence_embedding_dimension(self):
        """Returns the dimension of the final sentence embeddings after pooling and projection."""
        return self.proj_size # Return the projection size

# ------------------------
# Loss: multi-positive NT-Xent
# ------------------------

def multi_positive_nt_xent(embeddings: torch.FloatTensor,
                            metadata: List[dict],
                            temperature: float = 0.05,
                            device='cuda'):
    """
    embeddings: (M, D) where M is number of texts in the batch tokenization (including anchor copies and neighbors)
    metadata: list per anchor with positions where anchor copies and neighbor positions are located inside embeddings
    We will treat for each anchor:
      anchor_pos is index i, positives are anchor_pos2 and any neighbor positions.
    Compute loss: -log( sum_{p in P(i)} exp(sim(i,p)/temp) / sum_{k != i} exp(sim(i,k)/temp) )
    and average over anchors. Also consider the symmetric direction? We'll average over anchors only (anchors are half of examples).
    """
    device = embeddings.device
    # sim = torch.matmul(embeddings, embeddings.T) # (M, M) cosine because embeddings are normalized
    # Compute cosine similarity matrix
    sim = F.cosine_similarity(embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=2)

    sim = sim / temperature

    M = sim.shape[0]
    losses = []
    # large_neg_mask = torch.eye(M, dtype=torch.bool, device=device) # remove self-sim
    exp_sim = torch.exp(sim)

    # Create a mask to exclude self-similarity
    self_mask = torch.eye(M, dtype=torch.bool, device=device)

    for m, meta in enumerate(metadata):
        i = meta['anchor_pos']
        positives = [meta['anchor_pos2']] + meta['neighbor_positions'] # positions of positives

        if len(positives) == 0:
            # This case should ideally not happen if anchor_pos2 is always included,
            # but as a safeguard, if somehow there are no positives, skip this anchor
            # or handle it as an error. Given the dataset/collate, anchor_pos2 should exist.
            # If num_neighbors can be 0 and sample_neighbors_prob can be 0, then positives could be empty.
            # For SimCSE-like unsupervised, anchor_pos2 is always a positive.
            # Let's ensure anchor_pos2 is always in positives.
             if meta['anchor_pos2'] not in positives:
                 positives.append(meta['anchor_pos2'])


        # numerator: sum over similarities with positives
        numer = exp_sim[i, positives].sum()

        # denominator: sum over all except i itself
        # Use the self_mask to exclude sim(i, i)
        denom = exp_sim[i].masked_fill(self_mask[i], 0.0).sum()

        # safety
        denom = denom.clamp(min=1e-9)

        loss_i = -torch.log(numer / denom)
        losses.append(loss_i)

    if not losses:
        # Handle case where no valid anchors were processed
        return torch.tensor(0.0, device=device)

    loss = torch.stack(losses).mean()
    return loss

# ------------------------
# Training loop
# ------------------------

def train_contrastive(model: SentenceEmbedder,
                      tokenizer,
                      dataset: ContrastiveSentenceDataset,
                      corpus_obj: SentenceCorpus, # Pass the corpus object
                      device='cuda',
                      epochs: int = 1,
                      batch_size: int = 64,
                      lr: float = 2e-5,
                      max_length: int = 64,
                      temperature: float = 0.05,
                      eval_every: Optional[int] = None,
                      num_workers: int = 2):
    """
    Training driver for unsupervised contrastive learning.
    Uses the ContrastiveSentenceDataset and multi_positive_nt_xent loss.
    """
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    # Pass corpus_obj to the collate_fn
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=lambda x: collate_batch_fast(x, tokenizer, max_length, corpus_obj))
    pbar = tqdm(range(epochs * len(loader)), desc="Contrastive Train Steps")
    step = 0
    for epoch in range(epochs):
        for batch_tok, metadata in loader:
            # batch_tok: dict of tensors
            # metadata: list per anchor
            input_ids = batch_tok['input_ids'].to(device)
            attention_mask = batch_tok['attention_mask'].to(device)

            # Forward pass: SimCSE-style requires model.train() for dropout
            embeddings = model(input_ids=input_ids, attention_mask=attention_mask)

            # Compute loss
            loss = multi_positive_nt_xent(embeddings, metadata, temperature=temperature, device=device)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            # Optional: clip gradients
            # torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()

            pbar.set_postfix({'loss': loss.item()})
            pbar.update(1)
            step += 1

            # Optional: Evaluation during training
            # if eval_every is not None and step % eval_every == 0:
            #      print(f"\nEvaluating at step {step}...")
            #      # Implement evaluation logic here (e.g., using evaluate_similarity_pairs on a small test set)
            #      model.eval()
            #      # eval_results = ...
            #      # print(f"Evaluation results: {eval_results}")
            #      model.train() # Set back to train mode

        print(f"\nEpoch {epoch+1}/{epochs} finished.")
        # Optional: Save checkpoint after each epoch
        # model.save(os.path.join(SAVE_DIR, f"model_epoch_{epoch+1}"))


    pbar.close()
    return model

# Helper: faster collate that avoids repeated tokenization decode/encode
def collate_batch_fast(batch_items: List[dict], tokenizer, max_length: int, corpus: SentenceCorpus):
    """
    Build list of texts more efficiently without decode/encode roundtrip.
    """
    texts = []
    metadata = []
    for it in batch_items:
        a = it['anchor_text']
        anchor_pos = len(texts)
        texts.append(a)        # view 1
        anchor_pos2 = len(texts)
        texts.append(a)        # view 2
        neighbor_positions = []
        for flat_idx in it['neighbor_pos_indices']:
            neighbor_positions.append(len(texts))
            # Use the corpus object to get the sentence text
            texts.append(corpus.get_sentence(flat_idx))
        metadata.append({
            'anchor_idx': it['anchor_idx'],
            'anchor_pos': anchor_pos,
            'anchor_pos2': anchor_pos2,
            'neighbor_positions': neighbor_positions
        })
    tok = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    return tok, metadata


# ------------------------
# Evaluation utilities
# ------------------------

def embed_sentences(model: SentenceEmbedder, tokenizer, texts: List[str], device='cuda', batch_size=128, max_length=64):
    model.eval()
    all_embs = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            tok = tokenizer(batch_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt').to(device)
            embs = model(tok['input_ids'], tok['attention_mask'])
            all_embs.append(embs.cpu().numpy())
    return np.vstack(all_embs)

def evaluate_similarity_pairs(model: SentenceEmbedder, tokenizer, pairs: List[Tuple[str, str]], gold_scores: Optional[List[float]] = None):
    """
    Compute cosine similarity for pairs. If gold_scores provided, compute Spearman correlation.
    Intended for supervised evaluation sets like STS-B.
    """
    if not pairs:
        print("No pairs provided for similarity evaluation.")
        return [], None, None

    texts = []
    for a, b in pairs:
        texts.append(a)
        texts.append(b)

    # Ensure model and tokenizer are on the correct device
    model.to(model.device) # Use the model's internal device
    # Tokenizer does not have a device, but its output tensors will be moved later

    embs = embed_sentences(model, tokenizer, texts, device=model.device, batch_size=128) # Use model's device

    sims = []
    for i in range(0, len(texts), 2):
        a_emb = embs[i]
        b_emb = embs[i+1]
        # Compute cosine similarity manually for numpy arrays
        sim = float(np.dot(a_emb, b_emb) / (np.linalg.norm(a_emb) * np.linalg.norm(b_emb) + 1e-9))
        sims.append(sim)

    if gold_scores is not None and len(sims) == len(gold_scores):
        try:
            rho, p = spearmanr(sims, gold_scores)
            return sims, rho, p
        except ValueError as e:
             print(f"Could not compute Spearman correlation: {e}. Check if all score values are identical.")
             return sims, None, None
    elif gold_scores is not None and len(sims) != len(gold_scores):
         print(f"Warning: Number of computed similarities ({len(sims)}) does not match number of gold scores ({len(gold_scores)}). Skipping Spearman correlation.")
         return sims, None, None

    return sims, None, None

# Example usage / Putting it together - This part is moved to the main execution block in the other cell
# if __name__ == '__main__':
#     # Example minimal usage (use your own corpus)
#     model_name = 'bert-base-uncased'
#     device = 'cuda' if torch.cuda.is_available() else 'cpu'
#     tokenizer = AutoTokenizer.from_pretrained(model_name)

#     # Dummy toy corpus: list of documents, each is list of sentences
#     docs = [
#         ["This is the first sentence.", "Here is a nearby sentence.", "Another sentence in the same doc."],
#         ["Unrelated document starts here.", "More text in other doc.", "Even more text."]
#     ]
#     corpus = SentenceCorpus(docs)
#     dataset = ContrastiveSentenceDataset(corpus, neighbor_window=3, num_neighbors=1, sample_neighbors_prob=0.9)

#     model = SentenceEmbedder(model_name=model_name, pooling='mean', device=device)

#     # Train for 1 epoch on toy data (in real setup use larger batches, many epochs)
#     model = train_contrastive(model, tokenizer, dataset, corpus, device=device, epochs=1, batch_size=8, lr=2e-5, max_length=64, temperature=0.05)

#     # Example evaluation: build pairs (identical, neighbor, random)
#     pairs = []
#     gold = []  # 1 = similar, 0 = dissimilar (for simple ROC/accuracy), or continuous scores for Spearman
#     # identical
#     pairs.append((docs[0][0], docs[0][0])); gold.append(1.0)
#     # neighbor (distance < window)
#     pairs.append((docs[0][0], docs[0][1])); gold.append(1.0)
#     # unrelated
#     pairs.append((docs[0][0], docs[1][0])); gold.append(0.0)

#     sims, rho, p = evaluate_similarity_pairs(model, tokenizer, pairs, gold_scores=gold)
#     print("pair similarities:", sims)
#     if rho is not None:
#         print("Spearman rho:", rho, "p:", p)

In [ ]:
# Example usage with df_papers

model_name = 'bert-base-uncased'
# Use the device determined earlier in the notebook
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure df_papers is loaded
if 'df_papers' not in globals():
    print("Error: df_papers DataFrame not found. Please run the cell to load papers.parquet first.")
else:
    # Prepare data for ContrastiveSentenceDataset
    # Group sentences by document ID to reconstruct the document structure
    docs = []
    if 'doc_id' in df_papers.columns and 'sentence' in df_papers.columns:
        # Sort by doc_id and sent_index before grouping to ensure correct sentence order within docs
        df_sorted = df_papers.sort_values(by=['doc_id', 'sent_index']).reset_index(drop=True)
        grouped_docs = df_sorted.groupby('doc_id')['sentence'].apply(list).tolist()
        docs = grouped_docs
        print(f"Prepared {len(docs)} documents from df_papers.")
    else:
        print("Error: 'doc_id' or 'sentence' column not found in df_papers. Cannot structure data into documents for contrastive learning.")
        # As a fallback, treat each sentence as a single-sentence document
        if 'sentence' in df_papers.columns:
            docs = [[s] for s in df_papers['sentence'].tolist()]
            print(f"Treated each of {len(docs)} sentences as a single document (no 'doc_id' column found).")
        else:
             print("Error: 'sentence' column not found in df_papers. Cannot create dataset for training.")
             docs = [] # Set docs to empty list to prevent further execution


    if docs:
        # Create SentenceCorpus and ContrastiveSentenceDataset
        # Pass the DataFrame to SentenceCorpus for robust indexing
        corpus = SentenceCorpus(df_papers) # Use the updated SentenceCorpus that accepts DataFrame
        dataset = ContrastiveSentenceDataset(corpus, neighbor_window=10, num_neighbors=1, sample_neighbors_prob=0.9)
        print(f"Created ContrastiveSentenceDataset with {len(dataset)} examples.")

        # Instantiate the SentenceEmbedder model
        # You can specify proj_size if you want a different output dimension
        model = SentenceEmbedder(model_name=model_name, pooling='mean', device=device, proj_size=768) # Example with BERT base hidden size
        print(f"SentenceEmbedder model instantiated with output dimension: {model.get_sentence_embedding_dimension()}")


        # Train the model using the contrastive training function
        print("Starting contrastive training...")
        # Define training parameters (can be adjusted)
        epochs = 1
        batch_size = 32 # Adjust based on GPU memory
        lr = 2e-5
        max_length = 128 # Adjust based on sentence length in your data
        temperature = 0.05

        model = train_contrastive(model, tokenizer, dataset, corpus, device=device,
                                  epochs=epochs, batch_size=batch_size, lr=lr,
                                  max_length=max_length, temperature=temperature)

        print("Contrastive training finished.")

        # Save the model weights after training
        model_save_path = "./sentence_embedder_weights.pth"
        torch.save(model.state_dict(), model_save_path)
        print(f"Model weights saved to {model_save_path}")


        # Example evaluation after training (using a small subset)
        print("\nEvaluating on a small subset...")
        # You can create some example pairs from your df_papers data for evaluation
        # For intrinsic evaluation, you can use the PCA and Participation Ratio functions from the other cell.
        # Example: Embed a few sentences
        sentences_to_embed = df_papers['sentence'].tolist()[:10] # Embed first 10 sentences
        if sentences_to_embed:
            embeddings = embed_sentences(model, tokenizer, sentences_to_embed, device=device)
            print(f"Embeddings for first {len(sentences_to_embed)} sentences: shape {embeddings.shape}")
            # You can now use these embeddings for downstream tasks or further analysis
        else:
            print("No sentences to embed for evaluation.")


    else:
        print("Could not prepare data for training. Please check df_papers and column names ('doc_id', 'sentence').")

Prepared 1 documents from df_papers.
Created ContrastiveSentenceDataset with 32 examples.
SentenceEmbedder model instantiated with output dimension: 768
Starting contrastive training...


Contrastive Train Steps:   0%|          | 0/1 [00:00<?, ?it/s]


Epoch 1/1 finished.
Contrastive training finished.
Model weights saved to ./sentence_embedder_weights.pth

Evaluating on a small subset...
Embeddings for first 10 sentences: shape (10, 768)


In [ ]:
embeddings

array([[-0.01535399,  0.03278962, -0.02682263, ..., -0.01716761,
        -0.01679298,  0.05872576],
       [-0.00279186,  0.00956682, -0.03053568, ..., -0.02641026,
         0.03312619,  0.05788518],
       [ 0.02387265,  0.02183133, -0.0109611 , ..., -0.02721242,
        -0.01510445,  0.05418845],
       ...,
       [-0.02536347, -0.02762472, -0.01766055, ..., -0.00779511,
        -0.02404274,  0.01465486],
       [-0.04725742, -0.01500422,  0.02574473, ..., -0.0086509 ,
        -0.02516468,  0.01174189],
       [-0.05033306, -0.03705888,  0.02572924, ..., -0.00325346,
        -0.01694051,  0.04906924]], dtype=float32)